# OpenSearch 벡터 스토어 및 Amazon Bedrock를 사용한 RAG 
> 이 노트북은  SageMaker Studio* **`Data Science 3.0`** kernel 및 ml.t3.medium 인스턴스에서 테스트 되었습니다.
---
### 중요
- 이 노트북은 Anthropic 의 Claude-v2 모델 접근 가능한 분만 실행 가능합니다. 
- 접근이 안되시는 분은 노트북의 코드와 결과 만을 확인 하시면 좋겠습니다.
- 만일 실행시에는 **"과금"** 이 발생이 되는 부분 유념 해주시기 바랍니다.

### 선수조건
- 임베딩 모델의 세이지 메이커 엔드포인트가 액티브 된 상태를 가정 합니다.
    - 세이지 메이커 엔드포인트에 배포하기 위해서는 아래 노트북을 실행하시고, Endpoint Name 만을 복사 하시면 됩니다.
    - [KoSIMCSERoberta Embedding Model 배포](https://github.com/gonsoomoon-ml/Kor-LLM-On-SageMaker/blob/main/1-Lab01-Deploy-LLM/4.Kor-Embedding-Model.ipynb)
    - SageMaker Endpoint 에 대해서는 공식 개발자 문서를 참조하세요 --> [Create your endpoint and deploy your model](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-deployment.html)
- 오픈 서치 서비스가 액티브 된 상태를 가정 합니다.
---

OpenSearch는 대규모 데이터셋에 대한 유사도 검색을 위한 강력한 엔진입니다. Amazon OpenSearch Service를 통해 쉽게 클라우드 환경에서도 이용할 수 있습니다. 이와 함께 Vector Store를 사용하면 고차원 벡터 데이터를 효율적으로 저장하고 빠르게 검색할 수 있어, 복잡한 자연어 처리 작업을 더욱 간편하게 수행할 수 있습니다.


### 설정

이 노트북의 나머지 부분을 실행하기 전에 아래 셀을 실행하여 (필요한 라이브러리가 설치되어 있는지 확인하고) Bedrock에 연결해야 합니다.



In [3]:
install_needed = True  # should only be True once
# install_needed = False  # should only be True once

In [4]:
# Make sure you ran `download-dependencies.sh` from the root of the repository first!
import sys
import IPython

if install_needed:
    print("installing deps and restarting kernel")
    !{sys.executable} -m pip install -U pip
    !{sys.executable} -m pip install -U sagemaker    
    !{sys.executable} -m pip install opensearch-py==2.3.1
    !{sys.executable} -m pip install langchain==0.0.279    
    !{sys.executable} -m pip install sqlalchemy==2.0.1
    
    IPython.Application.instance().kernel.do_shutdown(True)
    

installing deps and restarting kernel


In [1]:
! pip list | grep langchain
! pip list | grep opensearch
! p[p list |  sqlalchemy

langchain                            0.0.279
opensearch-py                        2.3.1
/bin/bash: -c: line 1: unexpected EOF while looking for matching `]'
/bin/bash: -c: line 2: syntax error: unexpected end of file


In [2]:
%load_ext autoreload
%autoreload 2

import sys, os
module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import print_ww


# 1. Bedrock Client 생성

In [3]:
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
# os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

Create new client
  Using region: us-east-1
boto3 Bedrock client successfully created!
bedrock(https://bedrock.us-east-1.amazonaws.com)


# 2. Titan Embedding 및 LLM 인 Claude-v2 모델 로딩

## LLM 로딩 (Claude-v2)

In [4]:
from langchain.llms.bedrock import Bedrock

# - create the Anthropic Model
llm_text = Bedrock(model_id="anthropic.claude-v2", client=boto3_bedrock, 
              model_kwargs={'max_tokens_to_sample':512})
llm_text

Bedrock(cache=None, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, client=<botocore.client.Bedrock object at 0x7f5f5aa38b80>, region_name=None, credentials_profile_name=None, model_id='anthropic.claude-v2', model_kwargs={'max_tokens_to_sample': 512}, endpoint_url=None)

## Embedding 모델 선택

In [5]:
from utils.rag import KoSimCSERobertaContentHandler, SagemakerEndpointEmbeddingsJumpStart

def get_embedding_model(is_bedrock_embeddings, is_KoSimCSERobert, aws_region, endpont_name=None):
    if is_bedrock_embeddings:

        # We will be using the Titan Embeddings Model to generate our Embeddings.
        from langchain.embeddings import BedrockEmbeddings
        llm_emb = BedrockEmbeddings(client=boto3_bedrock)
        print("Bedrock Embeddings Model Loaded")
    elif is_KoSimCSERobert:
        LLMEmbHandler = KoSimCSERobertaContentHandler()
        endpoint_name_emb = endpont_name
        llm_emb = SagemakerEndpointEmbeddingsJumpStart(
            endpoint_name=endpoint_name_emb,
            region_name=aws_region,
            content_handler=LLMEmbHandler,
        )        
        print("KoSimCSERobert Embeddings Model Loaded")
    else:
        llm_emb = None
        print("No Embedding Model Selected")
    
    return llm_emb

#### [중요] is_KoSimCSERobert == True 일시에 endpoint_name 을 꼭 넣어 주세요.

In [6]:
is_bedrock_embeddings = True
# is_bedrock_embeddings = False
# is_KoSimCSERobert = True
is_KoSimCSERobert = False

##############################
# Parameters for is_KoSimCSERobert
##############################
aws_region = os.environ.get("AWS_DEFAULT_REGION", None)
endpont_name = "KoSimCSE-roberta-2023-09-01-14-34-11"
##############################

llm_emb = get_embedding_model(is_bedrock_embeddings, is_KoSimCSERobert, aws_region, endpont_name)    

Bedrock Embeddings Model Loaded


# 3. 데이터 준비


##  신한은행 FAQ 데이터 세트로 구현
- [중요] 저자 및 동료가 아래의 웹사이트에서 크로링한 기준으로 구성 하였습니다.
- 인터넷뱅킹 FAQ > 스마트뱅킹 No.1 ~ N. 89 로 구성되었습니다. 
- https://www.shinhan.com/hpe/index.jsp#050101020000

In [7]:
import pandas as pd
pd.options.display.max_rows = 20

data_file_path = "data/fsi_smart_faq_ko.csv"
df = pd.read_csv(data_file_path)
df

,no,Category,Information,type,Source
0,91,아마존 은행의 타기관OTP 이용등록방법 알려주세요,아마존 은행의 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 ...,인터넷뱅킹,아마존은행
1,90,아마존 공동인증서와 금융인증서 차이점이 무엇인가요?,공동인증서 (구 공인인증서)는 용도에 따라 은행/신용카드/보험용 무료 인증서와 전자...,인증서,아마존은행
2,88,공동인증서와 금융인증서 차이점이 무엇인가요?,공동인증서 (구 공인인증서)는 용도에 따라 은행/신용카드/보험용 무료 인증서와 전자...,인증서,신한은행
3,89,타기관OTP 이용등록방법 알려주세요,타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다....,인터넷뱅킹,신한은행
4,88,공동인증서와 금융인증서 차이점이 무엇인가요?,공동인증서 (구 공인인증서)는 용도에 따라 은행/신용카드/보험용 무료 인증서와 전자...,인증서,신한은행
...,...,...,...,...,...
87,5,인터넷에서 이체한 거래의 상세내역 및 영수증을 인쇄하고 싶어요,"인터넷상에서 이체한 거래의 경우 인터넷상에서 제공하는 영수증은 없으며, 거래의 참고...",인터넷뱅킹,신한은행
88,4,타행으로 자동이체를 신청하고 싶은데요,인터넷뱅킹을 통한 타은행 계좌로의 자동이체(납부자 자동이체)신청이 가능하며 수수료는...,인터넷뱅킹,신한은행
89,3,공과금 자동이체 신청이 가능한가요?,"인터넷뱅킹에 로그인하신 후 ""공과금/법원 > 공과금센터"" 페이지에 가시면 ""지로자동...",인터넷뱅킹,신한은행
90,2,인터넷뱅킹 거래에 대한 책임 범위를 알고싶습니다.,기본적으로 은행이 인터넷뱅킹 사고에 대한 책임을 집니다. 해커의 침입에 의해 자금이...,인터넷뱅킹,신한은행


## 데이터 전처리
- 여기서 no 는 제거 합니다. 

In [8]:
def preprocess_data(df):
    ldf = df.copy()

    ldf.rename(columns={'Category': 'ask'}, inplace=True)
    df_index = ldf.drop(['no'], axis=1)
    # df_index.to_csv("rag_data_kr/fsi_smart_faq_ko_answer.csv", index=None, header=None)
    df_index.to_csv("data/fsi_smart_faq_ko_preprocess.csv", index=None)

    return df_index

pre_df = preprocess_data(df)
pre_df.head(3)

,ask,Information,type,Source
0,아마존 은행의 타기관OTP 이용등록방법 알려주세요,아마존 은행의 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 ...,인터넷뱅킹,아마존은행
1,아마존 공동인증서와 금융인증서 차이점이 무엇인가요?,공동인증서 (구 공인인증서)는 용도에 따라 은행/신용카드/보험용 무료 인증서와 전자...,인증서,아마존은행
2,공동인증서와 금융인증서 차이점이 무엇인가요?,공동인증서 (구 공인인증서)는 용도에 따라 은행/신용카드/보험용 무료 인증서와 전자...,인증서,신한은행


### CSVLoader 로 문서 로딩

In [12]:
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import FAISS
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter

In [13]:
loader = CSVLoader(
    file_path="data/fsi_smart_faq_ko_preprocess.csv",
    # csv_args={
    #     "delimiter": ",",
    #     "fieldnames": ["Category", "Information", "type", "Source"],
    # },    
    source_column="Source",
    encoding="utf-8"
)


## 메타 데이터 생성
- 기존 Category 컬럼을 ask 로 변경 합니다.
- 컬럼의 type, source 는 metadata 로 생성하고, 내용에서는 삭제 합니다.
- 타임스탬프 및 임베딩 모델의 엔드포인트 이름을 metadata 로 추가 합니다.

In [14]:
import time
documents_fsi = loader.load()

def create_metadata(docs):
    # # add a custom metadata field, such as timestamp
    for idx, doc in enumerate(docs):
        # type 을 메타 데이타로 저장
        stype = doc.page_content.split("type: ")[1].split("\n")[0]
        # print("stype: ", stype)
        split_content = doc.page_content.split("type: ")
        content = split_content[0]        
        metadata = split_content[1]                
        doc.metadata['type'] = metadata.split("\n")[0]        
        doc.page_content = content # metadata 제외하고 content 만 저장
        # doc.metadata['type'] = doc.page_content.split("type: ")[1].split("\n")[0]
        doc.metadata['timestamp'] = time.time()
        # doc.metadata['embeddings_model'] = llm_emb.endpoint_name
        
create_metadata(documents_fsi)


In [15]:
print(len(documents_fsi))

documents_fsi[0]
# documents_fsi[-1]

92


Document(page_content='ask: 아마존 은행의 타기관OTP 이용등록방법 알려주세요\nInformation: 아마존 은행의 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다. \r\n[경로]\r\n- 인터넷뱅킹 로그인→ 사용자관리→인터넷뱅킹관리→OTP이용등록  \r\n- 아마존은행 쏠(SOL) 로그인→ 전체메뉴→설정/인증→ 이용중인 보안매체선택→   OTP이용등록\r\n \r\n ※ OTP이용등록후 재로그인을 하셔야 새로 등록된 보안매체가 적용됩니다.\r\n\r\n기타 궁금하신 내용은 아마존 은행 고객센터 1599-9999로 문의하여 주시기 바랍니다.\n', metadata={'source': '아마존은행', 'row': 0, 'type': '인터넷뱅킹', 'timestamp': 1694075630.5796695})

## Text Spliter 로 청킹
참고: 검색된 문서/텍스트는 질문에 대답하기에 충분한 정보를 포함할 만큼 커야 합니다. 하지만 LLM 프롬프트에 들어갈 만큼 충분히 작습니다. 또한 임베딩 모델에는 입력 토큰 길이가 512개 토큰으로 제한되어 있으며 이는 한국어의 경우에 대략 180 ~ 200 글자로 변환됩니다. 이 사용 사례를 위해 [RecursiveCharacterTextSplitter](https://python.langchain.com/en/latest/modules/indexes/text_splitters/examples/recursive_text_splitter.html)를 사용하여 210자가 겹치는 약 50자의 청크를 생성합니다.

In [16]:
if is_bedrock_embeddings:
    chunk_size = 230
    chunk_overlap = 0
elif is_KoSimCSERobert:
    chunk_size = 800 # This is maxumum
    chunk_overlap = 0
    

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = chunk_size,
    chunk_overlap  = chunk_overlap,
    separators=["\n\n", "\n", ".", " ", ""],
    length_function = len,
)

docs = text_splitter.split_documents(documents_fsi)
print(f"Number of documents after split and chunking={len(docs)}")

Number of documents after split and chunking=215


## 청킹 통계 및 내용 확인

In [17]:
from utils.rag import show_context_used, show_chunk_stat
    
show_chunk_stat(docs)

                0
count  215.000000
mean   133.302326
std     69.529841
min      1.000000
25%     66.000000
50%    147.000000
75%    193.500000
max    230.000000
Average length among 215 documents loaded is 133 characters.

Show document at maximum size
ask: 금융인증서 발급대상은 누구인가요?
Information: 금융인증서는 은행 등의 금융회사에서 실명 확인된 고객을 대상으로 안전하게 발급되며 신한은행 온라인서비스 가입 고객이면 발급가능합니다. 다만 금융인증서는 PC하드디스크나 휴대폰 등에 파일형태로 저장하는 방식이 아닌 금융결제원의 클라우드에 저장되기 때문에 금융결제원의 클라우드 연결을 위해 문자, ARS, 마이인포 앱을 통한 인증 절차가 진행됩니다.


In [18]:
show_context_used(docs, limit=5)

-----------------------------------------------
1. Chunk: 201 Characters
-----------------------------------------------
ask: 아마존 은행의 타기관OTP 이용등록방법 알려주세요
Information: 아마존 은행의 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다.
[경로]
- 인터넷뱅킹 로그인→ 사용자관리→인터넷뱅킹관리→OTP이용등록
- 아마존은행 쏠(SOL) 로그인→ 전체메뉴→설정/인증→ 이용중인 보안매체선택→   OTP이용등록
metadata:
 {'source': '아마존은행', 'row': 0, 'type': '인터넷뱅킹', 'timestamp': 1694075630.5796695}
-----------------------------------------------
2. Chunk: 93 Characters
-----------------------------------------------
※ OTP이용등록후 재로그인을 하셔야 새로 등록된 보안매체가 적용됩니다.

기타 궁금하신 내용은 아마존 은행 고객센터 1599-9999로 문의하여 주시기 바랍니다.
metadata:
 {'source': '아마존은행', 'row': 0, 'type': '인터넷뱅킹', 'timestamp': 1694075630.5796695}
-----------------------------------------------
3. Chunk: 191 Characters
-----------------------------------------------
ask: 아마존 공동인증서와 금융인증서 차이점이 무엇인가요?
Information: 공동인증서 (구 공인인증서)는 용도에 따라 은행/신용카드/보험용 무료 인증서와 전자거래범용(수수료 4,400원) 인증서가 있으며 유효기간은 1년입니다.
아마존 공동인증서는 하드디스크나 이동식디스크, 휴대폰 등 원하시는

# 4. OpenSearch 벡터 Indexer 생성
### 선수 조건
- 아래의 링크를 참조해서 OpenSearch Service 를 생성하고, opensearch_domain_endpoint, http_auth 를 복사해서, 아래 셀의 내용을 대체 하세요.
    - [OpenSearch 생성 가이드](https://github.com/gonsoomoon-ml/Kor-LLM-On-SageMaker/blob/main/2-Lab02-QA-with-RAG/4.rag-fsi-data-workshop/TASK-4_OpenSearch_Creation_and_Vector_Insertion.ipynb)
- 랭체인 오프서처 참고 자료
    - [Langchain Opensearch](https://python.langchain.com/docs/integrations/vectorstores/opensearch)

#### [중요] 아래에 OpenSearch ID/PW 를 입력을 해주세요.

In [19]:
os.environ["OpenSearch_UserName"] = "Type ID" 
os.environ["OpenSearch_UserPassword"] = "Type PW" 

rag_user_name = os.environ["OpenSearch_UserName"]
rag_user_password = os.environ["OpenSearch_UserPassword"]



In [20]:
# OpenSearch Dashboards URL = https://search-gonsoo-jeiuxgzz4322ulsrbokvnqispu.us-east-1.es.amazonaws.com/_dashboards
opensearch_domain_endpoint = "https://search-gonsoo-jeiuxgzz4322ulsrbokvnqispu.us-east-1.es.amazonaws.com"
http_auth = (rag_user_name, rag_user_password) # Master username, Master password

index_name = "fsi-v10"


## 오픈 서치 인덱스 유무에 따라 삭제
오픈 서치에 해당 인덱스가 존재하면, 삭제 합니다. 

In [21]:
from utils.rag import create_aws_opensearch_client, check_if_index_exists, delete_index

aws_client = create_aws_opensearch_client(
                                     aws_region,
                                     opensearch_domain_endpoint,
                                     http_auth)



index_exists = check_if_index_exists(aws_client, index_name)

if index_exists:
    delete_index(aws_client, index_name)    
else:
    print("Index does not exist")

index_name=fsi-v10, exists=True

Deleting index:
{'acknowledged': True}


In [22]:
%%time

# by default langchain would create a k-NN index and the embeddings would be ingested as a k-NN vector type
from langchain.vectorstores import OpenSearchVectorSearch
docsearch = OpenSearchVectorSearch.from_documents(
    index_name=index_name,
    documents=docs,
    embedding=llm_emb,
    opensearch_url=opensearch_domain_endpoint,
    http_auth=http_auth,
    bulk_size=10000,
    timeout=60
)

CPU times: user 1.09 s, sys: 89.5 ms, total: 1.18 s
Wall time: 34.7 s


## 인덱싱된 벡터 확인

- [langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch.html)

In [23]:
vectro_db = OpenSearchVectorSearch(
    index_name=index_name,
    opensearch_url=opensearch_domain_endpoint,
    embedding_function=llm_emb,
    http_auth=http_auth, # http_auth
    is_aoss =False,
    engine="faiss",
    space_type="l2"
)

## OpenSearch Dashboard 통한 Query 및 레코드 확인
- OpenSearch Dashboards URL 은 Amazon OpenSearch Servie 콘솔에 가시면 확인할 수 있습니다.
OpenSearch DevTools Console DSL 방법
- [OpenSearch_Query_DSL](https://opensearch.org/docs/latest/query-dsl/index/)

![record_opensearch](img/record_opensearch.jpg)

# 5.오픈 서치에 "유사 서치" 검색
- query 를 제공해서 실제로 유사한 내용이 검색이 되는지를 확인 합니다.



- similarity_search_with_score API 정보
    - [API: similarity_search_with_score](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch.html#langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch.similarity_search)

In [24]:
from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain

In [25]:
from utils.rag import opensearch_pretty_print_documents        

def run_similarity_search_with_score_filter(query, is_filter, boolean_filter, k=5):
    if is_filter:
        pre_similar_doc = vectro_db.similarity_search_with_score(
            query,
            k=k,
            # fetch_k=3,
            search_type="approximate_search", # approximate_search, script_scoring, painless_scripting
            space_type="l2",     #"l2", "l1", "linf", "cosinesimil", "innerproduct", "hammingbit";
            boolean_filter= boolean_filter
        )
    else:
        pre_similar_doc = vectro_db.similarity_search_with_score(
            query,
            k=k,
            # fetch_k=3,
            search_type="approximate_search", # approximate_search, script_scoring, painless_scripting
            space_type="l2",     #"l2", "l1", "linf", "cosinesimil", "innerproduct", "hammingbit";
        )
        
    
    return pre_similar_doc



## Filter 없이 실행. 

In [26]:
# query = '간편조회서비스는 회원가입해야하나요?'
query = "타기관OTP 등록 방법 알려주세요"

# filter01 = "공과금"

# filter01 = "인터넷뱅킹"
pre_similar_doc = run_similarity_search_with_score_filter(query, is_filter=False, boolean_filter=None, k=3)
opensearch_pretty_print_documents(pre_similar_doc)


Score: 0.010057886
Document Number: 0
Metadata:
Type: 인터넷뱅킹
Source: 아마존은행
--------------------------------------------------

Score: 0.008723928
Document Number: 3
ask: 타기관OTP 이용등록방법 알려주세요
Information: 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다. 
Metadata:
Type: 인터넷뱅킹
Source: 신한은행
--------------------------------------------------

Score: 0.008668
Document Number: 11
ask: 인터넷뱅킹에서 모바일OTP 인증요청 했는데 푸시메세지가 오지 않아요. 어떻게 해야하나요?
Information: 신한 쏠(SOL) 푸시알림 수신에 동의를 하신경우 푸시를 받을수 있습니다. 경로 
Metadata:
Type: 인터넷뱅킹
Source: 신한은행
--------------------------------------------------


## metadata의 type, source 에 필터를 걸어서 검색

In [27]:
def create_bool_filter(filter01, filter02):
    boolean_filter = {
        "bool": {
          "must": [
            {
              "match": {
                "metadata.type": f"{filter01}"
              }
            }
          ],
          "filter": {
            "term": {
              "metadata.source": f"{filter02}"
            }
          }
        }
    }
    
    return boolean_filter

In [28]:
filter01 = "인터넷뱅킹"
# filter01 = "인증서"
filter02 = "신한은행"
# filter02 = "아마존은행"

boolean_filter = create_bool_filter(filter01, filter02)
pre_similar_doc = run_similarity_search_with_score_filter(query, is_filter=True, boolean_filter=boolean_filter, k=10)
opensearch_pretty_print_documents(pre_similar_doc)


Score: 0.008723928
Document Number: 3
ask: 타기관OTP 이용등록방법 알려주세요
Information: 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다. 
Metadata:
Type: 인터넷뱅킹
Source: 신한은행
--------------------------------------------------

Score: 0.008668
Document Number: 11
ask: 인터넷뱅킹에서 모바일OTP 인증요청 했는데 푸시메세지가 오지 않아요. 어떻게 해야하나요?
Information: 신한 쏠(SOL) 푸시알림 수신에 동의를 하신경우 푸시를 받을수 있습니다. 경로 
Metadata:
Type: 인터넷뱅킹
Source: 신한은행
--------------------------------------------------

Score: 0.008423055
Document Number: 12
ask: 모바일 OTP 사용하고 있는데 인터넷뱅킹에서는 어떻게 이용하는건가요?
Information: 인터넷뱅킹 거래시 보안매체 입력단계에 [인증요청] 버튼 클릭하면 핸드폰으로 푸시메세지가 전달됩니다. 
Metadata:
Type: 인터넷뱅킹
Source: 신한은행
--------------------------------------------------

Score: 0.008007645
Document Number: 33
Metadata:
Type: 인터넷뱅킹
Source: 신한은행
--------------------------------------------------

Score: 0.0074054287
Document Number: 12
Metadata:
Type: 인터넷뱅킹
Source: 신한은행
--------------------------------------------------

Score: 0.0067972443
Document Number: 71
Metadata:
Type: 인터

# 6.사용자 정의 가능한 옵션
이제 벡터 저장소가 준비되었으므로 질문을 시작할 수 있습니다.

Vector Store를 둘러싸서 LLM 입력을 받는 LangChain에서 제공하는 래퍼를 사용할 수 있습니다.
이 래퍼는 뒤에서 다음 단계를 수행합니다.
- 질문을 입력합니다.
- 질문 임베딩 생성
- 관련 문서 가져오기
- 프롬프트에 문서와 질문을 채워 넣습니다.
- 프롬프트로 모델을 호출하고 사람이 읽을 수 있는 방식으로 답변을 생성합니다.

위 시나리오에서는 질문에 대한 상황 인식 답변을 빠르고 쉽게 얻을 수 있는 방법을 탐색했습니다. 이제 문서를 가져오는 방법을 사용자 정의할 수 있는 [RetrievalQA](https://python.langchain.com/en/latest/modules/chains/index_examples/Vector_db_qa.html)의 도움으로 더 사용자 정의 가능한 옵션을 살펴보겠습니다. `chain_type` 매개변수를 사용하여 프롬프트에 추가해야 합니다. 또한 검색해야 하는 관련 문서 수를 제어하려면 아래 셀에서 'k' 매개변수를 변경하여 다른 출력을 확인하세요. 많은 시나리오에서 LLM이 답변을 생성하는 데 사용한 소스 문서가 무엇인지 알고 싶을 수 있습니다. LLM 프롬프트의 컨텍스트에 추가된 문서를 반환하는 `return_source_documents`를 사용하여 출력에서 ​​해당 문서를 가져올 수 있습니다. 'RetrievalQA'를 사용하면 모델에 특정한 사용자 정의 [프롬프트 템플릿](https://python.langchain.com/en/latest/modules/prompts/prompt_templates/getting_started.html)을 제공할 수도 있습니다.

참고: 이 예에서는 Amazon Bedrock에서 LLM으로 Anthropic Claude를 사용하고 있습니다. 이 특정 모델은 입력이 'Human:' 아래에 제공되고 모델이 'Assistant:' 다음에 출력을 생성하도록 요청되는 경우 가장 잘 수행됩니다. 아래 셀에는 LLM이 기본 상태를 유지하고 컨텍스트 외부에서 응답하지 않도록 프롬프트를 제어하는 ​​방법의 예가 나와 있습니다.

In [29]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

prompt_template = """Human: 다음 문맥의 Information을 사용하여 고객 서비스 센터 직원처럼, 마지막 질문에 대한 목차 형식으로 답변을 제공하세요. 읃답을 모르면 모른다고 말하고 응답을 만들려고 하지 마세요.

{context}

Question: {question}
Assistant:"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

def run_RetrievalQA(query, boolean_filter, llm_text, PROMPT, verbose, is_filter, k):
    if is_filter:
        qa = RetrievalQA.from_chain_type(
            llm=llm_text,
            chain_type="stuff",
            retriever=vectro_db.as_retriever(
                        search_type="similarity", 
                        search_kwargs={
                                        "k": k,
                                        "boolean_filter" : boolean_filter
                                    }
            ),

            return_source_documents=True,
            chain_type_kwargs={
                                "prompt": PROMPT,
                                "verbose": verbose,
                              },
            verbose=verbose
        )
    else:
        qa = RetrievalQA.from_chain_type(
            llm=llm_text,
            chain_type="stuff",
            retriever=vectro_db.as_retriever(
                        search_type="similarity", 
                        search_kwargs={
                                        "k": k
                                    }
            ),

            return_source_documents=True,
            chain_type_kwargs={
                                "prompt": PROMPT,
                                "verbose": verbose,
                              },
            verbose=verbose
        )
        
    result = qa(query)
    
    return result 

result = run_RetrievalQA(query, boolean_filter, llm_text, PROMPT, verbose=False, is_filter=True ,k=5)

print("##################################")
print("query: ", query)
print("##################################")
print_ww(result['result'])

##################################
query:  타기관OTP 등록 방법 알려주세요
##################################
 타기관 OTP를 신한은행에서 사용하시려면 다음과 같은 방법으로 등록하셔야 합니다.

1. 인터넷뱅킹 로그인
2. 사용자관리 > 인터넷뱅킹관리 > OTP이용등록

또는

1. 신한 쏠(SOL) 로그인
2. 전체메뉴 > 설정/인증 > 이용중인 보안매체선택 > OTP이용등록

OTP 등록 후 재로그인하셔야 새로 등록한 보안매체가 적용됩니다.

자세한 내용은 신한은행 고객센터(1599-8000)로 문의하시기 바랍니다.


In [30]:
show_context_used(result['source_documents'])        

-----------------------------------------------
1. Chunk: 228 Characters
-----------------------------------------------
ask: 타기관OTP 이용등록방법 알려주세요
Information: 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다.
[경로]
- 인터넷뱅킹 로그인→ 사용자관리→인터넷뱅킹관리→OTP이용등록
- 신한 쏠(SOL) 로그인→ 전체메뉴→설정/인증→ 이용중인 보안매체선택→   OTP이용등록

 ※ OTP이용등록후 재로그인을 하셔야 새로 등록된 보안매체가 적용됩니다.
metadata:
 {'source': '신한은행', 'row': 3, 'type': '인터넷뱅킹', 'timestamp': 1694075630.5796833}
-----------------------------------------------
2. Chunk: 225 Characters
-----------------------------------------------
ask: 인터넷뱅킹에서 모바일OTP 인증요청 했는데 푸시메세지가 오지 않아요. 어떻게 해야하나요?
Information: 신한 쏠(SOL) 푸시알림 수신에 동의를 하신경우 푸시를 받을수 있습니다. 경로 : 신한 쏠(SOL) 로그인 > 전체메뉴 > 설정/인증  > 일반 > 알림
> 금융정보 알림 항목을 수신동의로 설정 기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다.
metadata:
 {'source': '신한은행', 'row': 11, 'type': '인터넷뱅킹', 'timestamp': 1694075630.5797086}
-----------------------------------------------
3. Chunk: 182 Characters
-----------------------------------------------
ask: 모바일 O

In [31]:
query = "홈페이지 탈퇴하는방법 알려줘"
filter01 = "홈페이지"
# filter01 = "인증서"
filter02 = "신한은행"
# filter02 = "아마존은행"

boolean_filter = create_bool_filter(filter01, filter02)

result = run_RetrievalQA(query, boolean_filter, llm_text, PROMPT, verbose=False, is_filter=True ,k=3)

print("##################################")
print("query: ", query)
print("##################################")
print_ww(result['result'])

##################################
query:  홈페이지 탈퇴하는방법 알려줘
##################################
 정보를 바탕으로 홈페이지 탈퇴 방법에 대한 목차는 다음과 같습니다:

1. 홈페이지에 로그인한다.

2. 상단 메뉴의 고객센터 > 회원서비스 메뉴로 들어간다.

3. 회원탈퇴 메뉴를 선택한다.

4. 회원탈퇴 진행 절차에 따라 탈퇴를 완료한다.

5. 만약 비밀번호를 분실한 경우, ID/PW 로그인 화면 하단의 비밀번호 재설정을 통해 비밀번호를 변경한 후 탈퇴를 진행할 수 있다.

6. 기타 문의사항은 고객센터(☎1599-8000)로 문의할 수 있다.

이와 같은 절차를 통해 홈페이지 탈퇴가 가능합니다. 제공된 정보를 바탕으로 최대한 이해하기 쉽게 설명드렸습니다.


In [32]:
show_context_used(result['source_documents'])        

-----------------------------------------------
1. Chunk: 84 Characters
-----------------------------------------------
Information: 홈페이지 비밀번호를 5회 오류의 경우 홈페이지 로그인이 불가능하오니 본인인증 절차 후 비밀번호를 새로 등록하시면 이용 가능합니다
metadata:
 {'source': '신한은행', 'row': 59, 'type': '홈페이지', 'timestamp': 1694075630.5798566}
-----------------------------------------------
2. Chunk: 215 Characters
-----------------------------------------------
Information: 홈페이지 로그인 후 회원탈퇴를 하실 수 있습니다. 홈페이지 상단에 있는 고객센터 > 회원서비스 > 회원탈퇴 메뉴에서 회원탈퇴 진행할 수 있습니다. 혹시,
비밀번호를 분실하셨다면 ID/PW 로그인 화면 하단 비밀번호 재설정 메뉴를 통해서 비밀번호 재설정하고 로그인 후 회원탈퇴 가능합니다. 기타 문의는 신한은행고객센터 (☎
1599-8000번)으로 문의 바랍니다.
metadata:
 {'source': '신한은행', 'row': 44, 'type': '홈페이지', 'timestamp': 1694075630.579815}
-----------------------------------------------
3. Chunk: 230 Characters
-----------------------------------------------
. 신한은행 홈페이지 접속 후 ① (오른쪽 상단)고객센터 회원서비스 ② 회원정보변경 비밀번호 찾기(재설정) ③ (개인회원)약관보기 및 동의 ④ 본인확인(이름, 생년월일, 아이디,
계좌번호, 계좌비밀번호 입력) ⑤ 사용할 비밀번호/비밀번호 확인 2회 입력확인 ※ 영문, 숫자, 특수문자

In [33]:
query = "홈페이지 이용자아이디 여러 개 사용할 수 있나요?"
filter01 = "홈페이지"
# filter01 = "인증서"
filter02 = "신한은행"
# filter02 = "아마존은행"

boolean_filter = create_bool_filter(filter01, filter02)

result = run_RetrievalQA(query, boolean_filter, llm_text, PROMPT, verbose=False, is_filter=True ,k=3)

print("##################################")
print("query: ", query)
print("##################################")
print_ww(result['result'])

##################################
query:  홈페이지 이용자아이디 여러 개 사용할 수 있나요?
##################################
 홈페이지 이용자아이디 사용 관련해서는 다음과 같은 내용을 안내드릴 수 있습니다.

1. 개인의 경우 1인 1개의 이용자아이디만 사용 가능합니다.

2. 기업의 경우 1개 사업자번호당 사용자별로 이용자아이디를 복수로 사용할 수 있습니다.

3. 개인사업자는 개인과 기업으로 각각 이용자아이디를 발급받아 복수 사용이 가능합니다.

4. 기타 자세한 문의는 콜센터(1599-8000)로 문의하시기 바랍니다.

이용자아이디 사용 관련 상세내용은 위 내용을 참고하시기 바라며, 추가 궁금한 점이 있으시면 콜센터로 문의주시기 바랍니다.


In [34]:
show_context_used(result['source_documents'])        

-----------------------------------------------
1. Chunk: 182 Characters
-----------------------------------------------
ask: 홈페이지 이용자아이디 여러 개 사용할 수 있나요?
Information: 홈페이지 이용자 아이디는 개인의 경우 1인 1개만 이용 가능하고 기업의 경우에는 1개의 사업자번호당 사용자별로 이용자아이디를 복수로 사용할 수 있습니다.
※ 개인사업자의 경우 개인과 기업 각각 이용자아이디를 발급하여 복수로 이용 가능합니다.
metadata:
 {'source': '신한은행', 'row': 21, 'type': '홈페이지', 'timestamp': 1694075630.5797412}
-----------------------------------------------
2. Chunk: 182 Characters
-----------------------------------------------
. [구비서류] - 법정대리인 실명확인증표 - 미성년자 기준으로 발급된 특정 또는 상세 기본증명서 - 미성년자 기준으로 발급된 가족관계증명서 * 은행 방문일로부터 3개월 이내
발급한 서류로 준비해주셔야 하며, 성함과 주민등록번호 13자리가 모두 기재되어 있어야합니다. 기타 문의는 콜센터 1599-8000번으로 문의 바랍니다.
metadata:
 {'source': '신한은행', 'row': 48, 'type': '홈페이지', 'timestamp': 1694075630.5798259}
